In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchWindowException, NoSuchElementException

import pandas as pd
import time

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

In [6]:
driver = webdriver.Chrome(executable_path="C:/Users/Allen/GitHub/DrexelCiProject/chromedriver.exe")

<ipython-input-6-529689e1e99e>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/Allen/GitHub/DrexelCiProject/chromedriver.exe")


In [7]:
def scrape_from_individual_job_url(link: str):
    time.sleep(5)
    driver.get(link)
    
    job_title = driver.find_element(By.CLASS_NAME, "icl-u-xs-mb--xs.icl-u-xs-mt--none.jobsearch-JobInfoHeader-title").text

    data = driver.find_element(By.CLASS_NAME, "icl-u-xs-mt--xs.icl-u-textColor--secondary.jobsearch-JobInfoHeader-subtitle.jobsearch-DesktopStickyContainer-subtitle")
    data_list = data.text.split('\n')
    company_name = data_list[0]
    company_location = data_list[-1]
    
    job_description = driver.find_element(By.CLASS_NAME, "jobsearch-jobDescriptionText").text
    job_activity = driver.find_element(By.CLASS_NAME, "jobsearch-HiringInsights-entry--text").text
    
    try:
        benefits = driver.find_elements(By.CLASS_NAME, "css-tvvxwd.ecydgvn1")
        benefits_list = [benefit.text for benefit in benefits]    
    except:
        benefits_list = None
        
    return job_title, company_name, company_location, job_description, job_activity, benefits_list

In [11]:
def scrape_all_job_data(job_title_to_search: str, city: str):    
    job_title_to_search = job_title_to_search.replace(" ", "+")
    url_template = "https://www.indeed.com/jobs?q={}&l={}".format(job_title_to_search, city)

    driver.get(url_template)
    original_page = driver.current_url
    original_window = driver.current_window_handle

    href_elements = driver.find_elements(By.CLASS_NAME, "jcs-JobTitle.css-jspxzf")
    links = [elem.get_attribute('href') for elem in href_elements]
    
    job_titles, company_names, company_locations, job_descriptions, job_activities, benefits_lists = [], [], [], [], [], []
    
    driver.switch_to.new_window()
    tab_that_goes_to_individual_jobs = driver.current_window_handle
    
    for link in links:
        try:
            job_title, company_name, company_location, job_description, job_activity, benefits_list = scrape_from_individual_job_url(link)

            job_titles.append(job_title)
            company_names.append(company_name)
            company_locations.append(company_locations)
            job_descriptions.append(job_description)
            job_activities.append(job_activity)
            benefits_lists.append(benefits_list)
            
        except (NoSuchWindowException, NoSuchElementException):
            pass
        
    driver.switch_to.window(original_window)
    
    driver.find_element(By.XPATH, '//a[@aria-label="Next Page"]').click()
            
    scraped_data_df = pd.DataFrame(data = {"Job Title": job_titles, 
                                           "Company Name": company_names, 
                                           "Company Location": company_locations,
                                           "Job Description": job_descriptions,
                                           "Benefits": benefits_lists,
                                           "Job Posted Date": job_activities})
    
    
    return scraped_data_df

In [9]:
df = scrape_all_job_data("Data Scientist", "Philadelphia")

In [10]:
df.head()

,Job Title,Company Name,Company Location,Job Description,Benefits,Job Posted Date
0,Data Scientist,firstPRO Inc.,"[[[[[...], [...], [...], [...], [...], [...], ...",firstPRO inc. is looking for a Data Scientist ...,"[401(k), Dental insurance, Flexible schedule, ...",Fast hiring
1,Data Scientist-Syndicated Data Exp,"Task Management, Inc.","[[[[[...], [...], [...], [...], [...], [...], ...",Data Scientist-PHD students-Contract- HYBRID C...,[],Fast hiring
2,Data Scientist II,TEKsystems,"[[[[[...], [...], [...], [...], [...], [...], ...",Equivalent Experience\nTop Skills' Details Ski...,[],Posted 2 days ago
3,Data Scientist,"Worldgate, llc","[[[[[...], [...], [...], [...], [...], [...], ...",Job Summary:\nThe Data Analyst (DA) supports o...,"[401(k), Dental insurance, Health insurance, P...",Posted 16 days ago
4,Jr. Data Scientist,"Data-Core System, Inc.","[[[[[...], [...], [...], [...], [...], [...], ...",Data-Core Systems is looking for a Jr. Data Sc...,"[401(k), 401(k) matching, Dental insurance, He...",Posted 8 days ago


In [13]:
df = scrape_all_job_data("Software Engineer", "Philadelphia")

In [14]:
df.head()

,Job Title,Company Name,Company Location,Job Description,Benefits,Job Posted Date
0,"Software Engineer, Backend (Remote-eligible)",Capital One,"[[[[[...], [...], [...], [...], [...], [...], ...","Center 1 (19052), United States of America, Mc...",[],Posted 4 days ago
1,Embedded Systems Developer,Scala,"[[[[[...], [...], [...], [...], [...], [...], ...",Scala solutions deliver engaging retail experi...,"[401(k), 401(k) matching, Dental insurance, Di...",Posted 30+ days ago
2,Senior Application Software Developer,ASRC Federal Holding Company,"[[[[[...], [...], [...], [...], [...], [...], ...",ASRC Federal Mission Solutions is a premier pr...,[],Posted 30+ days ago
3,Software Engineer – Early Career,LOCKHEED MARTIN CORPORATION,"[[[[[...], [...], [...], [...], [...], [...], ...",COVID-19 continues to significantly impact our...,[Flexible schedule],Posted 30+ days ago
4,"Software Engineer, Full Stack",Capital One,"[[[[[...], [...], [...], [...], [...], [...], ...","1735 Market St (16035), United States of Ameri...",[],Posted today


In [92]:
# Page 1 start=0, Page2 start=10 ... Page 15 start=140

current_page = print(driver.current_url)

https://www.indeed.com/jobs?q=data+scientist&l=Philadelphia%2C+PA&start=80&pp=gQB4AAAAAAAAAAAAAAAB6Yk_wABaAQIBECIPA0jaRWN8J6g8JwqiagpbDsBMMTj2AdJMfyDZsMaIcNrx1xSOxIlOFiWmFT2AKBtF6MXf6vyOCTOWtG8WDm1q9Iyw1A4H3u2WkJCeKRfl2-Ww-N9JAAA&vjk=cadfde065bde5c0a


In [ ]:
# Number of jobs
<div class="jobsearch-JobCountAndSortPane-jobCount">

In [103]:
job_title = driver.find_element(By.CLASS_NAME, "icl-u-xs-mb--xs.icl-u-xs-mt--none.jobsearch-JobInfoHeader-title")
print(job_title.text)

Senior Data Scientist


In [93]:
driver.get('https://www.indeed.com/viewjob?jk=ecbcf343b3e3b8ce&tk=1gfjvhrjeg0n6802&from=serp&vjs=3&advn=4753071484186304&adid=359869966&ad=-6NYlbfkN0Cj-KmZPsf9w80C8b1WzNVrlanjD2SXJjxuCbUWHsXPZkFBy4Qr63BQKSyytxWB3SjjsGgK0lHsSMLiVZIKUrGNXwq5l1i7UNlTHr2wL3ADRozZFUrGyp2F9zy0vVoKkw89-RcQvfS-6RCKpWBUw1aEOnlaMpBO_cCixhqxF36vqnvnMcamHMkKARcxkgZDpxmtJVuOadNVlJ1O4-MQbR0msnurHQm3_vYm6azBsJbdDvQ0pAX6d68g-RaIYiCpxo1B5pAWEowdvKkVB6f_E_XQysHzbBBL511gg5yRHjTKyLaKeCvS6tI6Ue7T6kIGhTFkwxUjO7A6wjGPoc3esv4Ywli9A1S58DrPKbMV1rnq0dYBhQs59axT&sjdu=iMYN2Jly9sWyarqkEfXE5mk08BBOpcp3pYGzYADa6uVn7OC3DCWNiGDD5b1e2m2u2WJi_oaZ5ZeqGRnSSb1E0uTXjwX_1CW8-NhEEroXrMeyGmouCVyCauSCDVOG2g2tngKJn31rruz7NF1v4wPNSb5LDUCVObz6EAUsFekLErIpBJHaDsBLoJzh_X__sIWVI-zDlgpgF3wIMZanaHf28rApfn_Qx_WEgvZLmH_D2TfpAurgmrngMbUBeJA-VI12e7HFlosxcVUscCQmgoXPA6-1OCKfPzeoI9R1iY6aOfi2B4Nzs6CFo8N6ZuPsASj1eRKixf1VQqLQGo0CxEtWv73qCrFIBo3AuEc4mnCaZJLhHSCGQDbLZHnmCj0dRexGT3bfqTLONiZpZKzT7a41yZsY082_DSpTg9pIROgaL7dRSYbdN8ehgsYWosxq5_m-XowTNQFf7suJKsclP_UAygyLQzMytTZ1ja1hv9PiAOmy_LtdzpowrbxNaHvChHf-PoNUDV7KPMvt6bY4SA84nJRzlM013kjyswnNEDy-sB8zSDlgrtbbv8t7Uy9Ec8jsyG9gXKIdqZmHXu1xJZqn1A29iiZgUGDloMInzfiZRCQ')
data = driver.find_element(By.CLASS_NAME, "icl-u-xs-mt--xs.icl-u-textColor--secondary.jobsearch-JobInfoHeader-subtitle.jobsearch-DesktopStickyContainer-subtitle")
print(data.text)

Comcast
17,839 reviews
Philadelphia, PA 19103


In [109]:
x = data.text.split('\n')
print(x[0])

Comcast


In [99]:
benefits = driver.find_elements(By.CLASS_NAME, "css-tvvxwd.ecydgvn1")
print(benefits[3].text)

Health insurance


In [101]:
job_activity = driver.find_element(By.CLASS_NAME, "jobsearch-HiringInsights-entry--text")
print(job_activity.text)

Posted 30+ days ago


In [22]:
job_title, company_name, company_location, job_description, job_activity, benefits_list = scrape_from_individual_job_url("")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".icl-u-xs-mt--xs.icl-u-textColor--secondary.jobsearch-JobInfoHeader-subtitle.jobsearch-DesktopStickyContainer-subtitle"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C81ED3+2236115]
	Ordinal0 [0x00C192F1+1807089]
	Ordinal0 [0x00B266FD+812797]
	Ordinal0 [0x00B555DF+1005023]
	Ordinal0 [0x00B557CB+1005515]
	Ordinal0 [0x00B87632+1209906]
	Ordinal0 [0x00B71AD4+1120980]
	Ordinal0 [0x00B859E2+1202658]
	Ordinal0 [0x00B718A6+1120422]
	Ordinal0 [0x00B4A73D+960317]
	Ordinal0 [0x00B4B71F+964383]
	GetHandleVerifier [0x00F2E7E2+2743074]
	GetHandleVerifier [0x00F208D4+2685972]
	GetHandleVerifier [0x00D12BAA+532202]
	GetHandleVerifier [0x00D11990+527568]
	Ordinal0 [0x00C2080C+1837068]
	Ordinal0 [0x00C24CD8+1854680]
	Ordinal0 [0x00C24DC5+1854917]
	Ordinal0 [0x00C2ED64+1895780]
	BaseThreadInitThunk [0x74148744+36]
	RtlGetAppContainerNamedObjectPath [0x76F2582D

In [182]:
job_title

'Senior Data Scientist'

In [183]:
company_name

'Comcast'

In [184]:
company_location

'Philadelphia, PA 19103'

In [185]:
job_description

"Comcast’s Technology, Product & Experience organization works at the intersection of media and technology. Our innovative teams are continually developing and delivering products that transform the customer experience. From creating apps like TVGo to new features such as the Talking Guide on the X1 platform, we work every day to make a positive impact through innovation in the pursuit of building amazing products that are enjoyable, easy to use and accessible across all platforms. The team also develops and supports our evolving network architecture, including next-generation consumer systems and technologies, infrastructure and engineering, network integration and management tools, and technical standards. We offer a flexible working environment to balance the need to work independently wherever you choose, with days that require collaboration at one of our offices. Our roles primarily reside inside or around one of our Tech Hubs (Philadelphia and surrounding suburbs, Denver, Austin,

In [186]:
job_activity

'Posted 30+ days ago'

In [187]:
benefits_list

['401(k)',
 'Adoption assistance',
 'Dental insurance',
 'Health insurance',
 'Paid time off',
 'Pet insurance']

In [ ]:
# Copied from this stackoverflow link. Maybe can fix? https://stackoverflow.com/questions/51775122/nosuchwindowexception-no-such-window-window-was-already-closed-while-switchi

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

url = "your_url"
driver = webdriver.Chrome(executable_path = r'S:\Engineering\Jake\MasterControl Completing Pipette CalPM Forms\chromedriver')
driver.get(url)
windows_before  = driver.current_window_handle
driver.find_element_by_id('portal.scheduling.prepopulate').click()
WebDriverWait(driver, 5).until(EC.number_of_windows_to_be(2))
windows_after = driver.window_handles
new_window = [x for x in windows_after if x != windows_before][0]
# driver.switch_to_window(new_window) <!---deprecated>
driver.switch_to.window(new_window)
WebDriverWait(driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.ID,"myframe"))) # or By.NAME, By.XPATH, By.CSS_SELECTOR

In [221]:
df = scrape_all_job_data("Data Scientist", "Philadelphia")

<ipython-input-220-b293d0871049>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/Allen/Desktop/chromedriver.exe")


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x01031ED3+2236115]
	Ordinal0 [0x00FC92F1+1807089]
	Ordinal0 [0x00ED66FD+812797]
	Ordinal0 [0x00EBDFFA+712698]
	Ordinal0 [0x00F2506B+1134699]
	Ordinal0 [0x00F3514A+1200458]
	Ordinal0 [0x00F218A6+1120422]
	Ordinal0 [0x00EFA73D+960317]
	Ordinal0 [0x00EFB71F+964383]
	GetHandleVerifier [0x012DE7E2+2743074]
	GetHandleVerifier [0x012D08D4+2685972]
	GetHandleVerifier [0x010C2BAA+532202]
	GetHandleVerifier [0x010C1990+527568]
	Ordinal0 [0x00FD080C+1837068]
	Ordinal0 [0x00FD4CD8+1854680]
	Ordinal0 [0x00FD4DC5+1854917]
	Ordinal0 [0x00FDED64+1895780]
	BaseThreadInitThunk [0x77938744+36]
	RtlGetAppContainerNamedObjectPath [0x77D2582D+253]
	RtlGetAppContainerNamedObjectPath [0x77D257FD+205]


In [219]:
df.head()

,Job Title,Company Name,Company Location,Job Description,Benefits,Job Posted Date
